# Lab 01 — Beacon Hunt

Find periodic destination endpoints in Zeek `conn.log`.


## Warm-up
- Inspect `logs/zeek/conn.log` (TSV).
- Columns are defined after a `#fields` header.


In [ ]:
import pandas as pd
from pathlib import Path

conn_path = Path('/home/jovyan/logs/zeek/conn.log')

# Parse Zeek TSV with header lines starting '#'
with conn_path.open() as f:
    lines = [ln for ln in f if not ln.startswith('#')]

# Get fields from header
with conn_path.open() as f:
    hdr = [ln for ln in f if ln.startswith('#fields')][0].strip().split('\t')[1:]

df = pd.DataFrame([ln.rstrip('\n').split('\t') for ln in lines], columns=hdr)
# Cast types where needed
for col in ('ts','id.resp_p'):
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Build endpoint string and compute inter-arrival deltas
import numpy as np

if len(df) == 0:
    print('No records found in conn.log')
else:
    df['endpoint'] = df['id.resp_h'] + ':' + df['id.resp_p'].astype(int).astype(str)
    out = []
    for ep, grp in df.sort_values('ts').groupby('endpoint'):
        ts = grp['ts'].astype(float).values
        if len(ts) < 2:
            continue
        diffs = np.diff(ts)
        mean = float(np.mean(diffs))
        std = float(np.std(diffs))
        cv = (std/mean) if mean > 0 else 999.0
        out.append((ep, len(ts), mean, std, cv))
    cand = pd.DataFrame(out, columns=['endpoint','count','period_mean','period_std','cv']).sort_values(['cv','count'], ascending=[True,False])
    cand.head(10)


## Write answers
Fill the `answers` dict below using your findings, then write `answers.yml` at the lab root.


In [ ]:
import yaml
answers = {
  'beacons': [
    {'endpoint': '203.0.113.55:443', 'period_sec': 60, 'count': 10},
    {'endpoint': '198.51.100.9:80',  'period_sec': 30, 'count': 12},
    {'endpoint': '203.0.113.99:22',  'period_sec': 45, 'count': 8},
  ]
}
with open('/home/jovyan/labs/01_beacon/answers.sample.yml', 'w') as f:
    yaml.safe_dump(answers, f, sort_keys=False)
print('Wrote /home/jovyan/labs/01_beacon/answers.sample.yml')
